In [2]:
from pulp import *
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt
os.chdir('/Users/chengchen/Desktop/Insight/project/')
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Optimization of Locations for Charging Stations
* Objective: minimizing total cost (cost of installing chargers + cost of travelling between charge point and destination)
* Demand at all places need to be satisfied.
* Charging capacity is not surpassed at each charging station. <br><br>
* The optimization problem is summarized as: 
$$\min \sum^m_{j=1}f_j y_j + \sum^n_i \sum^m_j c_{ij}x_{ij}$$
$$ s.t.\ \sum^m_{j=1}x_{ij} = d_i,\ \ \ \ i = 1,...,n$$
$$ \ \ \ \ \ \sum^n_{i=1}x_{ij} \leqslant M_j y_j, \ \ \ \ j = 1,...,m$$
$$ \ \ \ \ \ x_{ij}\leqslant d_i y_j, \ \ \ \ i = 1,...,n$$
$$ \ \ \ x_{ij}\geqslant 0 $$
$$ \ \ \ y_j \in \{0,\ 1\}$$
where $f_j$ = fixed cost of charging station at location $j$,  
$y_j \in \{0,1\}$ indicates whether to build a charging station at location $j$,  
$c_{ij}$ is the travel cost for person at desired destination location $i$ to charging station at location $j$,  
$x_{ij}$ is the demand of charging by person at location $i$ that will be covered by charging station at location $j$,  
$d_i$ is demand of charging at destination location $i$.

### Sets

In [32]:
# sets of charging demand location and charging station location candidates
df_demand = pd.read_excel('data/cleaned/optimization_demand.xlsx')
df_chg = pd.read_excel('data/cleaned/optimization_chg_lc.xlsx').set_index('id')

In [33]:
demand_lc = df_demand.id.tolist()

In [34]:
len(demand_lc)

510

In [36]:
chg_lc = df_chg.index.tolist()
len(chg_lc)

428

### Parameters

#### demand at each demand point (centroids of Census Tracts in City of Toronto)
* $d_i = N\_Trip_i \times EVPR \times Prob(needs\ charging)$  
* $d_i$ is Demand for charging at point $i$  ($i=1,2,...,510$ represents centroids of Census Tracts in City of Toronto);  
* EVPR is (projected) Electric Vehicle Penetration Rate in City of Toronto;  
* Prob(needs charging) is the probability of an EV that needs to be charged using public charging stations, which is * further determined by (1)long or short commute and (2)whether having access to home charging.


In [54]:
# EVPR and Prob(needs charging) will be determined by user-input parameters
# for now, start with something simple and try whether the algorithm works

In [38]:
def gen_demand(df, EVPR, home_chg_ratio, long_commute_ratio):
    """generate the demand for charging (during day time) for each census tract"""
    pr_ch = long_commute_ratio * 1 + (1-long_commute_ratio)*(1-home_chg_ratio)/5
    # assumptions here: (1) ppl with longer commute (from outside of the city) always need public charging
    #                   (2) ppl with short commute only need public charging (1/5 of the times) if they do not have access to home charging
    df['demand_chg'] = df['CT_AM_trips']*EVPR*pr_ch
    return df
EVPR = 0.01
home_chg_ratio = 0.8
long_commute_ratio = 0.1
df_demand2 = gen_demand(df_demand, EVPR, home_chg_ratio, long_commute_ratio)

In [40]:
df_demand2.demand_chg.sum() # under current setting, 1337 cars need to be charged per day

1337.2580936215486

In [41]:
demand = df_demand2['demand_chg'].to_dict()

#### Cost of each charging station
Assume each charging station has 5 level II chargers, the cost of installing the chargers in the parking lot is approximately 11000$.  
https://www.homeadvisor.com/cost/garages/install-an-electric-vehicle-charging-station/#level2

In [42]:
df_chg['fixed_cost'] = 11000
fixed_cost = df_chg['fixed_cost'].to_dict()

#### Charging Capacity of each charging station
Here we only consider the situation of level II charging which is the most common public charging type. It can fully charge an electric car within 4-6 hours. It is ideal to use this type of charging service while working or doing some other activities nearby during day time. The full capacity of each charging station with 5 level II chargers would be fully charging 10 cars.

In [44]:
df_chg['chg_capacity'] = 10
capacity = df_chg['chg_capacity'].to_dict()

#### Cost of driving/walking from/to charging stations
This cost will be proportional to distance between charging station location and charging demand (destination) location.

In [48]:
# distance matrix of charging station location candidates and charging demand location
df_distance = pd.read_excel('data/cleaned/distance_mtx.xlsx')
df_travel_cost = df_distance * 2 
# currently assumed 2$/1km, this parameter also needs justification, also, it could be a nonlinear relationship (never go too far)
dic_cost_matrix = df_travel_cost.to_dict('index')

### Problem setup
#### Set problem variable

In [51]:
prob = LpProblem('FacilityLocation', LpMinimize)

#### Set decision variables

In [52]:
serv_vars = LpVariable.dicts("Service",
                             [(i,j) for i in demand_lc
                                    for j in chg_lc],
                              0)
use_vars = LpVariable.dicts("UseLocation", chg_lc, 0, 1, LpBinary)

#### Objective function
* Objective: minimizing total cost (cost of installing chargers + cost of travelling between charge point and destination)  

In [53]:
prob += lpSum(fixed_cost[j]*use_vars[j] for j in chg_lc) + lpSum(dic_cost_matrix[j][i]*serv_vars[(i,j)] for j in chg_lc for i in demand_lc)


#### Constraints
* Demand at all places need to be satisfied.
* Charging capacity is not surpassed at each charging station. 


In [55]:
for i in demand_lc:
    prob += lpSum(serv_vars[(i,j)] for j in chg_lc) == demand[i] # constraint 1
for j in chg_lc:
    prob += lpSum(serv_vars[(i,j)] for i in demand_lc) <= capacity[j]*use_vars[j]
for i in demand_lc:
    for j in chg_lc:
        prob += serv_vars[(i,j)] <= demand[i]*use_vars[j]


#### Solve

In [56]:
prob.solve()
print("Status: ", LpStatus[prob.status])

KeyboardInterrupt: 